# Agent-Powered Retrieval with Haystack

In this notebook, you'll build an intelligent **movie recommendation assistant** powered by **Haystack** and [**Qdrant**](https://haystack.deepset.ai/integrations/qdrant-document-store). You'll learn how to combine **sparse vector search**, **metadata filtering (payload)**, and **LLM-based agents** to create a system that can understand natural language queries and recommend relevant movies from a curated dataset.

By the end of this notebook, you’ll have a fully working assistant that can answer queries like:

* “Find me a highly-rated action movie about car racing.”
* “Can you recommend five japanese thrillers?”
* “What can I watch with my kids, about animals?”

This assistant will be implemented as a **tool-calling agent** and have access to a `retrieval_tool` that can retrieve the information from the database based on the **generated query** and [**filters**](https://docs.haystack.deepset.ai/docs/metadata-filtering).

## Haystack 🤝 Qdrant
[Haystack](https://haystack.deepset.ai/) is an open source AI framework that empowers Python developers to build **real-world, compound, agentic LLM** applications. It provides a modular, pipeline-based architecture that connects components like retrievers, generators, and agents, allowing you to create production-ready systems with minimal effort.

By abstracting away infrastructure complexity and providing intuitive interfaces, Haystack allows you to focus on the logic and behavior of your application instead of plumbing together low-level components.

## 🧱 Setting up the Development Environment

Install required dependencies:
* `haystack-ai`: For building pipelines and agents
* `datasets`: For loading the movie dataset from Hugging Face
* `qdrant-haystack`: For vector database access
* `fastembed-haystack`: For using FastEmbed library for sparse embedding generation

In [1]:
!pip install -q -U haystack-ai qdrant-haystack fastembed-haystack datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 592.1/592.1 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Add credentials for [OpenAI API](https://platform.openai.com/api-keys) and [Qdrant Cloud](https://qdrant.tech/documentation/cloud/authentication/)

In [2]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["QDRANT_API_KEY"] = userdata.get('QDRANT_API_KEY')

## 🎬 Loading Movie Dataset

Load the [Pablinho/movies-dataset](https://huggingface.co/datasets/Pablinho/movies-dataset) from Hugging Face

In [3]:
from datasets import load_dataset

data = load_dataset('Pablinho/movies-dataset', split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

9000plus.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/9837 [00:00<?, ? examples/s]

In [4]:
data[14]

{'Release_Date': '2022-01-28',
 'Title': 'The Ice Age Adventures of Buck Wild',
 'Overview': "The fearless one-eyed weasel Buck teams up with mischievous possum brothers Crash & Eddie as they head off on a new adventure into Buck's home: The Dinosaur World.",
 'Popularity': 1431.307,
 'Vote_Count': '737',
 'Vote_Average': '7.1',
 'Original_Language': 'en',
 'Genre': 'Animation, Comedy, Adventure, Family',
 'Poster_Url': 'https://image.tmdb.org/t/p/original/zzXFM4FKDG7l1ufrAkwQYv2xvnh.jpg'}

### Converting to Haystack Documents

Convert all movies into Haystack [Document](https://docs.haystack.deepset.ai/docs/data-classes#document) objects. `Document` dataclass holds text, tables, and binary data as content alongside with metadata.

In [5]:
from haystack.dataclasses import Document

def movie_to_document(movie_dataset):
    docs = []
    for movie in movie_dataset:
        # Process votes
        vote_avg_raw = movie.get('Vote_Average')
        try:
            rating = float(vote_avg_raw)
        except (TypeError, ValueError):
            rating = 5.0 # average value

        # Process genre
        genre_raw = movie.get("Genre")
        if isinstance(genre_raw, str):
            genre_list = [g.lower() for g in genre_raw.split(", ")]
        else:
            genre_list = []

        # Create Document
        doc = Document(
            content=movie.get('Overview', ''),
            meta={
                'title': movie.get('Title', ''),
                'rating': rating,
                'language': movie.get('Original_Language', ''),
                'genre': genre_list,
            }
        )
        docs.append(doc)
    return docs

In [6]:
documents = movie_to_document(data)

In [7]:
documents[14]

Document(id=1f5260fdf65717876901d115502f2d3286cc76ba582a05ca38249b16fb7423d7, content: 'The fearless one-eyed weasel Buck teams up with mischievous possum brothers Crash & Eddie as they he...', meta: {'title': 'The Ice Age Adventures of Buck Wild', 'rating': 7.1, 'language': 'en', 'genre': ['animation', 'comedy', 'adventure', 'family']})

## 📄  Creating Sparse Embeddings

Use [Qdrant/minicoil-v1](https://huggingface.co/Qdrant/minicoil-v1) sparse neural embedding model with `FastembedSparseDocumentEmbedder` component. We'll use `meta_fields_to_embed` parameter to include `title` metadata into the content.
After initializing `FastembedSparseDocumentEmbedder`, run `.warm_up()` to load the embedding model and get it ready to run. Then, run the component with documents to create embedding for each document.

In [8]:
from haystack_integrations.components.embedders.fastembed import FastembedSparseDocumentEmbedder

sparse_doc_embedder = FastembedSparseDocumentEmbedder(model="Qdrant/minicoil-v1", meta_fields_to_embed=["title"])
sparse_doc_embedder.warm_up()

sparse_embedder_result = sparse_doc_embedder.run(documents=documents)

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

stopwords.txt:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

minicoil.triplet.model.vocab: 0.00B [00:00, ?B/s]

minicoil.triplet.model.npy:   0%|          | 0.00/157M [00:00<?, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

Calculating sparse embeddings: 100%|██████████| 9837/9837 [03:08<00:00, 52.15it/s]


In [9]:
sparse_embedder_result["documents"][14]

Document(id=1f5260fdf65717876901d115502f2d3286cc76ba582a05ca38249b16fb7423d7, content: 'The fearless one-eyed weasel Buck teams up with mischievous possum brothers Crash & Eddie as they he...', meta: {'title': 'The Ice Age Adventures of Buck Wild', 'rating': 7.1, 'language': 'en', 'genre': ['animation', 'comedy', 'adventure', 'family']}, sparse_embedding: vector with 71 non-zero elements)

## ☁️ Writing Movies into Qdrant Cloud


Initialize the [QdrantDocumentStore](https://docs.haystack.deepset.ai/docs/qdrant-document-store) to connect to your Qdrant Cloud cluster.

You'll need to provide the following:

* `url`: The endpoint of your Qdrant Cloud instance
* `index`: The name of the index where documents will be stored
* `api_key`: Your Qdrant API key (set this securely via environment variables)

Make sure to set:

* `use_sparse_embeddings`=True to enable sparse vector support
* `payload_fields_to_index` to define which metadata fields (like genre, rating, language) are indexed for filtering

This setup ensures your vector store supports both sparse retrieval and metadata filtering, which are essential for advanced search scenarios.


In [12]:
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack.utils import Secret

document_store = QdrantDocumentStore(
    url="https://4135560a-7773-4253-9fa8-70415820592b.us-west-1-0.aws.cloud.qdrant.io:6333",
    index="movie-assistant",
    api_key=Secret.from_env_var("QDRANT_API_KEY"),
    use_sparse_embeddings=True,
    recreate_index=True,
    payload_fields_to_index=[
        {"field_name":"meta.language", "field_schema":"keyword"},
        {"field_name":"meta.rating", "field_schema":"float"},
        {"field_name":"meta.genre", "field_schema":"keyword"},
        {"field_name":"meta.title", "field_schema":"keyword"}
    ]
)


Now, call `.write_documents()` with the `Document` list.

In [13]:
document_store.write_documents(documents=sparse_embedder_result["documents"])

9900it [00:22, 441.78it/s]


9829

## 🔍 Creating a Sparse Retrieval Pipeline

Build a sparse retrieval pipeline that accepts a user query about what type of movie that they're looking for and retrieves relevant movie from our database.

First, initialize and add a [FastembedSparseTextEmbedder](https://docs.haystack.deepset.ai/docs/fastembedsparsetextembedder) with the `Qdrant/minicoil-v1` model to our pipeline. This component will convert user queries into sparse vector representations. We then add a [QdrantSparseEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/qdrantsparseembeddingretriever), which connects to our `QdrantDocumentStore` and uses those vectors to retrieve the top 5 most relevant movies.

Finally, we connect the two components by passing the output from the embedder into the retriever's input. This creates a functional query pipeline that transforms a natural language input into a sparse embedding and returns matching results from Qdrant.

In [14]:
from haystack_integrations.components.retrievers.qdrant import QdrantSparseEmbeddingRetriever
from haystack_integrations.components.embedders.fastembed import FastembedSparseTextEmbedder
from haystack import Pipeline

sparse_query_pipeline = Pipeline()
sparse_query_pipeline.add_component("sparse_text_embedder", FastembedSparseTextEmbedder(model="Qdrant/minicoil-v1"))
sparse_query_pipeline.add_component("sparse_retriever", QdrantSparseEmbeddingRetriever(document_store=document_store, top_k=5))
sparse_query_pipeline.connect("sparse_text_embedder.sparse_embedding", "sparse_retriever.query_sparse_embedding")

🚅 Components
  - sparse_text_embedder: FastembedSparseTextEmbedder
  - sparse_retriever: QdrantSparseEmbeddingRetriever
🛤️ Connections
  - sparse_text_embedder.sparse_embedding -> sparse_retriever.query_sparse_embedding (SparseEmbedding)

Run the pipeline with a query

In [15]:
sparse_docs = sparse_query_pipeline.run({"text":"A movie about car race"})

Calculating sparse embeddings: 100%|██████████| 1/1 [00:00<00:00, 172.81it/s]


See the retrieved movies:

In [16]:
for d in sparse_docs["sparse_retriever"]["documents"]:
  print("Title:", d.meta["title"])
  print("Content:", d.content)
  print("Rating:", d.meta["rating"])
  print("Genres:", d.meta["genre"])
  print("---")

Title: Cannonball Run II
Content: The original characters from the first Cannonball Run movie compete in an illegal race across the country once more in various cars and trucks.
Rating: 5.5
Genres: ['action', 'comedy']
---
Title: Ford v Ferrari
Content: American car designer Carroll Shelby and the British-born driver Ken Miles work together to battle corporate interference, the laws of physics, and their own personal demons to build a revolutionary race car for Ford Motor Company and take on the dominating race cars of Enzo Ferrari at the 24 Hours of Le Mans in France in 1966.
Rating: 8.0
Genres: ['drama', 'action', 'history']
---
Title: Driven
Content: Talented rookie race-car driver Jimmy Bly has started losing his focus and begins to slip in the race rankings. It's no wonder, with the immense pressure being shoveled on him by his overly ambitious promoter brother as well as Bly's romance with his arch rival's girlfriend Sophia. With much riding on Bly, car owner Carl Henry brings fo

### 🗑️ Adding Filters

Let's enrich the user query by combining semantic search with filters.

The text query "Car race" is first converted into a sparse embedding to make a search in movie descriptions. In addition, we apply two filters: the movie must have a rating of at least 7, and its genre must include "action". By combining both semantic similarity and structured filtering, we ensure that the results are not only topically relevant but also meet specific quality and genre constraints.

In [17]:
sparse_docs = sparse_query_pipeline.run({
    "sparse_text_embedder": {"text": "A movie about car race"},
    "sparse_retriever": {"filters":{
        "operator": "AND",
        "conditions": [
            {"field":"meta.rating", "operator":">=", "value":7},
            {"field":"meta.genre", "operator": "==", "value": "action"},
        ],
    }}
})

Calculating sparse embeddings: 100%|██████████| 1/1 [00:00<00:00, 236.82it/s]


In [18]:
for d in sparse_docs["sparse_retriever"]["documents"]:
  print("Title:", d.meta["title"])
  print("Content:", d.content)
  print("Rating:", d.meta["rating"])
  print("Genres:", d.meta["genre"])
  print("---")

Title: Ford v Ferrari
Content: American car designer Carroll Shelby and the British-born driver Ken Miles work together to battle corporate interference, the laws of physics, and their own personal demons to build a revolutionary race car for Ford Motor Company and take on the dominating race cars of Enzo Ferrari at the 24 Hours of Le Mans in France in 1966.
Rating: 8.0
Genres: ['drama', 'action', 'history']
---
Title: New Initial D the Movie - Legend 2: Racer
Content: The planned film trilogy retells the beginning of the story from Shuuichi Shigeno's original car-racing manga. High school student Takumi Fujiwara works as a gas station attendant during the day and a delivery boy for his father's tofu shop during late nights. Little does he know that his precise driving skills and his father's modified Toyota Sprinter AE86 Trueno make him the best amateur road racer on Mt. Akina's highway. Because of this, racing groups from all over the Gunma prefecture issue challenges to Takumi to se

## 📐 Filtering without Query

Initialize a metadata-only retriever ([FilterRetriever](https://docs.haystack.deepset.ai/docs/filterretriever)) that allows filtering movies based on fields like rating, language, or genre without semantic matching.

In [19]:
from haystack.components.retrievers.filter_retriever import FilterRetriever

filter_retriever = FilterRetriever(document_store=document_store)

Run `FilterRetriever` to get all movies rated 9.0 and above:

In [20]:
filter_retriever.run({
        "field":"meta.rating","operator":">=","value":9.0
    })

{'documents': [Document(id=743bbc48462d81f13673ec0557325cac8dd3230b5553f0769d5a009be6a8b9d1, content: 'Mexican stand-up comedian Franco Escamilla draws his jokes from real-life experiences -- and he's wi...', meta: {'title': 'Franco Escamilla: Por La Anécdota', 'rating': 9.2, 'language': 'es', 'genre': ['comedy']}, sparse_embedding: vector with 96 non-zero elements),
  Document(id=dadb030fd584ac9f86788a5f2c624cbdbd65986cc26fc87be8fee246f7893f67, content: 'The young and vigorous Huo Yuanjia was only focused on "subjugating people by force", and therefore ...', meta: {'title': 'Kung Fu Master Huo Yuanjia', 'rating': 10.0, 'language': 'zh', 'genre': ['action', 'drama']}, sparse_embedding: vector with 145 non-zero elements),
  Document(id=6ed0a2849029dac85b17dc885b2d6d1d1a106957e3f22ca80e6def22e32034f8, content: 'The story of chaos both body and mind occurs when "Brick" (Ton Nam - Piem Chon) receives a package o...', meta: {'title': 'My Sex Doll', 'rating': 9.0, 'language': 'th', 'genre': 

## 🤖 Building the Movie Recommendation Agent
In this section, we bring everything together by building a movie recommendation agent that leverages tools, a system prompt, and an LLM.

### 🛠️ Retrieval Tool
To interact with the data, the agent uses a retrieval tool. Every tool in Haystack requires a **name**, a **description**, a **function to call**, and a **parameter schema**.

We start by defining a retrieval function that dynamically chooses between semantic search and metadata-based filtering, depending on the input. If the user provides a query, the tool uses the sparse retrieval pipeline. If only filters are provided, it falls back to a metadata filter retriever. This logic gives the agent the flexibility to handle a wide range of user queries—from simple genre filters to more nuanced, semantic prompts.

In [21]:
from typing import Optional, Dict, Any, List

def retrieval_function(
    query: Optional[str] = None,
    metadata_filters: Optional[Dict[str, Any]] = None,
    top_k: Optional[int] = 5
) -> List[Document]:

    retrieved_documents = []
    if not query and not metadata_filters:
        raise ValueError("At least one of 'query' or 'metadata_filters' must be provided.")

    # Retrieve documents based on the input provided
    if query:
        retrieved_documents = sparse_query_pipeline.run({
            "text": query,
            "filters": metadata_filters, # Can include metadata filters in semantic query
            "top_k": top_k
        })["sparse_retriever"]
    # If there's only `metadata_filters`
    else:
        retrieved_documents = filter_retriever.run(filters=metadata_filters)
        if len(retrieved_documents.get("documents")) > top_k:
            retrieved_documents["documents"] = retrieved_documents["documents"][:top_k]

    return retrieved_documents

Define the JSON schema for tool parameters, including `query`, `top_k`, and `metadata_filters`, to validate and structure tool calls.

In [22]:
parameters = {
  "type": "object",
  "properties": {
    "top_k": {
      "type": "integer",
      "description": "number of movies to get",
      "default": 5,
    },
    "query": {
      "type": "string",
      "description": "query to retrieve movies based on semantic similarity of the description"
    },
    "metadata_filters": {
      "type": "object",
      "description": "metadata filters to filter the movies based on user's preference",
      "properties": {
        "operator": {
          "type": "string",
          "enum": ["AND", "OR", "NOT"],
          "default": "AND",
          "description": "operator to combine the conditions"
        },
        "conditions": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "field": {
                "type": "string",
                "enum": ["meta.title", "meta.rating", "meta.genre", "meta.language"]
              },
              "operator": {
                "type": "string",
                "enum": ["==", "!=", ">", "<", ">=", "<=", "not in"]
              },
              "value": {
                  "oneOf": [
                  { "type": "string" },
                  { "type": "integer" },
                  {
                    "type": "array",
                    "items": {"type": "string"}
                  }
                ]
              }
            },
            "required": ["field", "operator", "value"],
            "additionalProperties": False
          }
        }
      },
      "required": ["operator", "conditions"],
      "additionalProperties": False
    }
  },
  "additionalProperties": False,
}


Format the output of retrieved movies into readable text, including the title, content, rating, genre, and language. Then, wrap the retrieval function as a Haystack [`Tool`](https://docs.haystack.deepset.ai/docs/tool), specifying how it should be invoked by the agent and how outputs should be displayed.

In [23]:
from haystack.tools import Tool

def movie_to_string(documents) -> str:
    result_str = ""
    for document in documents:
        result_str += f"Movie details for {document.meta['title']}:\n{document.content}\nRating:{document.meta['rating']}\nGenres:{document.meta['genre']}\nLanguage:{document.meta['language']}\n---\n"
    return result_str

retrieval_tool = Tool(
    name="retrieval_tool",
    description="Use this tool to get movies fitting to the user criteria",
    function=retrieval_function,
    parameters=parameters,
    outputs_to_string={"source": "documents", "handler": movie_to_string}
)

### 🧠 Agent

First, you need to define a system prompt for the agent. This prompt instructs how to interpret user queries, generate tool inputs, and handle fallback strategies.

In [24]:
prompt = """
You are a movie retrieval assistant that helps users find movies from a database using `retrieval_tool`. Users describe what they are looking for in natural language.

Each movie entry in the database contains the following fields:

* `content`: *str* — movie description
* `meta.title`: *str* — movie title
* `meta.rating`: *float* — rating score (e.g., 8.3)
* `meta.genre`: *List[str]* — genres (e.g., `["drama", "thriller"]`)
* `meta.language`: *str* — original language (options are: 'bn', 'te', 'pt', 'ml', 'ja', 'it', 'lv', 'ar', 'da', 'no', 'hu', 'la', 'el', 'he', 'fi', 'ta', 'sv', 'ms', 'sr', 'ro', 'ru', 'nb', 'nl', 'id', 'cs', 'fr', 'th', 'tl', 'de', 'uk', 'et', 'es', 'pl', 'fa', 'is', 'eu', 'tr', 'cn', 'zh', 'en', 'ko', 'ca', 'hi'

### Task

Your goal is to generate an input for the `retrieval_tool` using the user's query, which may include:

* `query`: a natural language string to match against the `content` field (movie description) — optional
* `metadata_filters`: dictionary of filtering conditions using fields in the `meta.*` namespace — optional
* `top_k`: number of movies to retrieve — default is 5 unless the user specifies otherwise

**Important:** At least one of `query` or `metadata_filters` **must** be included. Use the most appropriate one based on the user request.

### Generation Guidelines

- Use `metadata_filters` when the user specifies in the query:
  * **Genres** (e.g., "I'd like to watch an action movie", "A romance movie...")
  * **Rating thresholds** (e.g., "at least 7.5", "above 6", "a good movie", "quality movie")
  * **Language constraints** (e.g., "only English movies")

- Use `query` when the user describes:
  * **Plot or story themes** (e.g., "a movie about friendship and betrayal")
  * **Emotions or tone** (e.g., "feel-good", "dark comedy")
  * **Character types or settings** (e.g., "set in space", "female-led action")

- Use both `query` and `metadata_filters` if there is enough information in the user query for both `query` and `metadata_filters`.
- If the user asks for a specific number of movies, set that value as `top_k`.
- If the user only specifies a genre, it's acceptable to return only `metadata_filters` — no query needed.
- All metadata values must be lowercase
- If required, you can combine the conditions with "AND", "NOT" or "OR" operators.

### Examples

Here are the tool call parameters for user queries:

* User: "Can you recommned horror movies with bad ratings?"
```json
{
  "metadata_filters": {
    "operator": "AND",
    "conditions": [
      {"field": "meta.genre", "operator": "==", "value": "horror"},
      {"field": "meta.rating", "operator": "<", "value": 5}
    ]
  }
}
```

* User: "I want a powerful courtroom drama, something emotionally intense, high rated"
```json
{
  "query": "powerful courtroom drama, emotionally intense",
  "metadata_filters": {
    "operator": "AND",
    "conditions": [
      {"field": "meta.genre", "operator": "==", "value": "drama"},
      {"field": "meta.rating", "operator": ">=", "value": 7}
    ]
  }
}
```

* User: "Give me five Japanese thrillers."
```json
{
  "top_k": 5,
  "metadata_filters": {
    "operator": "AND",
    "conditions": [
      {"field": "meta.language", "operator": "==", "value": "ja"},
      {"field": "meta.genre", "operator": "==", "value": "thriller"}
    ]
  }
}
```

* User: "I'd like to watch an action movie that is not drama"
```json
{
  "metadata_filters": {
    "operator": "AND",
    "conditions": [
      {"field": "meta.genre", "operator": "==", "value": "action"},
      {"field": "meta.genre", "operator": "not in", "value": ["drama"]}
    ]
  }
}
```

#### Fallback Strategy
If the search space is too narrow (i.e., few or no results), relax the filters and/or simplify the query to allow broader but still relevant results. Aim to preserve the user’s intent while increasing recall.
Don't ask for confirmation from the user. Just tell that you broadened the search with the result and return the latest list.
"""

Initialize the Haystack [`Agent`](https://docs.haystack.deepset.ai/docs/agent) with the retrieval tool and the system prompt. This Agent can understand user queries and respond with movie recommendations.

In [25]:
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage

movie_agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4o-mini"),
    system_prompt=prompt,
    tools=[retrieval_tool]
)

Send a sample query to the Agent. The Agent interprets the intent, invokes the retrieval tool, and returns suitable movie recommendations.

In [26]:
query = "Find me a highly-rated action movie about car racing."
# query = "What can I watch with my kids, about animals?"
# query = "Can you recommend five japanese thrillers?"
# query = "I'd like to watch a german action movie but I don't like thriller and horror"
# query = "I love Titanic. Which movie should I watch next? It should be a highly rated movie though"

messages = movie_agent.run([ChatMessage.from_user(query)])

Calculating sparse embeddings: 100%|██████████| 1/1 [00:00<00:00, 198.07it/s]


Print `messages["last_message"].text` to see the Agent's final response

In [27]:
print(messages["last_message"].text)

Here are some highly-rated action movies about car racing:

1. **Ford v Ferrari**
   - **Rating:** 8.0
   - **Genres:** Drama, Action, History
   - **Description:** American car designer Carroll Shelby and the British-born driver Ken Miles work together to build a revolutionary race car for Ford Motor Company and take on the dominating race cars of Enzo Ferrari at the 24 Hours of Le Mans in France in 1966.
   - **Language:** English

2. **New Initial D the Movie - Legend 2: Racer**
   - **Rating:** 8.1
   - **Genres:** Animation, Drama, Action
   - **Description:** High school student Takumi Fujiwara discovers his precise driving skills while working as a gas station attendant and becomes the best amateur road racer on Mt. Akina's highway.
   - **Language:** Japanese

3. **Watch Out, We're Mad**
   - **Rating:** 7.5
   - **Genres:** Action, Comedy
   - **Description:** After a tied 1st place in a local stunt race, two drivers start a contest to decide who will own a dune buggy. When a 

### 🔉 Enable Streaming

Enable streaming output by passing `print_streaming_chunk` as the `streaming_callback`. This shows real-time updates inlcuding all tool invocations as the agent processes the query.

In [28]:
from haystack.components.generators.utils import print_streaming_chunk

query = "Find me a highly-rated action movie about car racing."
# query = "What can I watch with my kids, about animals?"
# query = "Can you recommend five japanese thrillers?"
# query = "I'd like to watch a german action movie but I don't like thriller and horror"
# query = "I love Titanic. Which movie should I watch next? It should be a highly rated movie though"

messages = movie_agent.run([ChatMessage.from_user(query)], streaming_callback=print_streaming_chunk)

[TOOL CALL]
Tool: retrieval_tool 
Arguments: {"query":"highly-rated action movie about car racing","metadata_filters":{"operator":"AND","conditions":[{"field":"meta.genre","operator":"==","value":"action"},{"field":"meta.rating","operator":">=","value":7}]}}



Calculating sparse embeddings: 100%|██████████| 1/1 [00:00<00:00, 190.17it/s]


[TOOL RESULT]
Movie details for New Initial D the Movie - Legend 2: Racer:
The planned film trilogy retells the beginning of the story from Shuuichi Shigeno's original car-racing manga. High school student Takumi Fujiwara works as a gas station attendant during the day and a delivery boy for his father's tofu shop during late nights. Little does he know that his precise driving skills and his father's modified Toyota Sprinter AE86 Trueno make him the best amateur road racer on Mt. Akina's highway. Because of this, racing groups from all over the Gunma prefecture issue challenges to Takumi to see if he really has what it takes to be a road legend.
Rating:8.1
Genres:['animation', 'drama', 'action']
Language:ja
---
Movie details for New Initial D the Movie - Legend 1: Awakening:
The first movie in a trilogy, focusing on the battle against the Takahashi brothers. High school student Takumi Fujiwara works as a gas station attendant during the day and a delivery boy for his father's tofu sho